# Retrieval Augmented Generation (RAG)

This notebook demonstrates an example of using [LangChain](https://www.langchain.com/) to delvelop a Retrieval Augmented Generation (RAG) pattern. It uses Azure AI Document Intelligence as document loader, which can extracts tables, paragraphs, and layout information from pdf, image, office and html files. The output markdown can be used in LangChain's markdown header splitter, which enables semantic chunking of the documents. Then the chunked documents are indexed into Azure AI Search vectore store. Given a user query, it will use Azure AI Search to get the relevant chunks, then feed the context into the prompt with the query to generate the answer.

![Semantic chunking in RAG](../media/semantic-chunking-rag.png)


## Prerequisites
- An Azure AI Document Intelligence resource in one of the 3 preview regions: **East US**, **West US2**, **West Europe** - follow [this document](https://learn.microsoft.com/azure/ai-services/document-intelligence/create-document-intelligence-resource?view=doc-intel-4.0.0) to create one if you don't have.
- An Azure AI Search resource - follow [this document](https://learn.microsoft.com/azure/search/search-create-service-portal) to create one if you don't have.
- An Azure OpenAI resource and deployments for embeddings model and chat model - follow [this document](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal) to create one if you don't have.

## Setup

In [1]:
# pip install python-dotenv langchain langchain-community langchain-openai langchainhub openai tiktoken azure-ai-documentintelligence azure-identity azure-search-documents==11.4.0b8

In [2]:
"""
This code loads environment variables using the `dotenv` library and sets the necessary environment variables for Azure services.
The environment variables are loaded from the `.env` file in the same directory as this notebook.
"""
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
doc_intelligence_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")

In [3]:
from langchain import hub
from langchain_openai import AzureChatOpenAI
from doc_intelligence_loader import AzureAIDocumentIntelligenceLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch

## Load a document and split it into semantic chunks

In [4]:
# Initiate Azure AI Document Intelligence to load the document. You can either specify file_path or url_path to load the document.
loader = AzureAIDocumentIntelligenceLoader(file_path="data/BD-D100_D120GV_XGV.pdf", api_key = doc_intelligence_key, api_endpoint = doc_intelligence_endpoint, api_model="prebuilt-layout", table_max_rows=3)
docs = loader.load()

## split it into semantic chunks

In [5]:
# Split the document into chunks base on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

docs_string = docs[0].page_content
splits = text_splitter.split_text(docs_string)

tables = docs[1:]
print("Length of splits: " + str(len(splits)))

Length of splits: 107


## Embed and index the chunks

In [6]:
# Embed the splitted documents and insert into Azure Search vector store

aoai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2024-02-15-preview",  # e.g., "2023-12-01-preview"
)

vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")

index_name: str = "washer-index-mod"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=aoai_embeddings.embed_query,
)

In [7]:
vector_store.add_documents(documents=splits)

['MzM5MjVlYjUtNTE5Yy00YWM2LWFiM2QtY2Q5MDU3YTcxNTQy',
 'Yzg1MzU5ZWEtY2JjZi00MTc2LWIxMTktMDFkNDljZWNjYTBk',
 'N2Q0MDJhZjgtOWM5NS00NmNmLTk1N2ItZGQyY2U2ZmZkZDky',
 'NDdiNzZkNzYtZjYyMS00MzFhLThlNWMtNzcyNjUzYTk4ZTdk',
 'MDA2OGE4NzEtOTE4NS00ODEwLTk1YzktNGNlZjg4N2VkMzI1',
 'MWEwYmRjOWItODVlZC00ODNkLWE4ODQtZDJiZmRlM2JmZTkz',
 'ZjVhNDIyYjgtZGMxMC00MGEyLTg5YmUtNTlkNzZmNTM1NzNm',
 'OTg3NGM5NTItMGI5MS00ZDU3LTkxNTEtMTdlODYwNjg0Y2Jl',
 'MGY4YjZiYjQtMGQzMi00NzFhLWFkZmMtOTA0YjNjYWFkOGZj',
 'YmMxMTMxNzktMjJjYy00ZTE0LWIxNDItYTA5NTU1MGE1MTk2',
 'Y2ZiNDJkMWMtZjFkMS00MDhmLWIyZDktNTY4YTAzNmQ3ZmQy',
 'ZGYyM2YzZTYtYzRhNC00ZjA0LTk0MDMtZjZlZmJlZTMwYzYy',
 'MWUzMjdiNjEtOTYzMS00ODRjLTgzMzctOTJlMjYyMDcyOTE2',
 'Mjg3ZmU2OTYtYTc5MC00NDc2LTliOWItNDllYzcxMmJmOGY5',
 'N2ZmZTA2ZTktMzE1Mi00ZmIwLTliZmYtYWQ4MTgzMmY0YWVl',
 'MjVlYmI5MmItZTE0OC00ODJlLTllMTMtNjZjNmU4ZjI5Yzhm',
 'YzYxOWIwMDEtZDIxNy00YWY5LWFjN2YtMmFiZmM2NGIyNmM0',
 'NGU1YWZjYTYtNzQ0My00NzNkLWFlMDctMmI1Zjk2ZWZjNzVm',
 'ZGNjMDViNDktYjk4My00ZjQ5LWIxZjEtYTE2ZTBjNDYw

In [8]:
vector_store.add_documents(documents=tables)

['MTFmYTY2NzgtZGU5Yi00MDM3LWFiY2ItODYyOWI3ZWIzZjRl',
 'MTI3NzQ0MGEtYTM5NC00NzRmLWI0MzYtZTU1YmIxNDc4MTcw',
 'ZmQ0ZTMzODQtMDQ1YS00OTA2LWIyN2YtYWY0ZGJiZGYzN2Uz',
 'YzViZDA3OTAtMjE5Yy00YjIzLTg5YmUtOWQ5NjU4NWQ1NWIw',
 'NGFhZjViZDYtOGYyNy00YzVhLTg5NmUtMTkwM2FlYTY5MTkx',
 'YThiMzExNDctYWY5ZC00NWNlLTk5NWItNDZkNTk0MGNkYTcx',
 'NjI2NWJhOGYtYzFkNy00N2E3LTg1ZWQtNzM3ODllZmUzMWEy',
 'N2RhZmEzYTUtZTVmNC00MTgwLWFkMGMtNTg4ZmRmZDhhMTIy',
 'OGRlNTc1OTktYjIwZC00NDAwLTlhNjYtYWExY2ZjOTBkNmU2',
 'YTRhNzhmOTItMDM2OS00YWVjLWIzZTctN2QzNDg4ZDBiNGJm',
 'YmRiNDhhNDYtOWQ1MS00MjNlLWExNDUtNTMzYTIzZjcwOWVi',
 'ZTZhMmUwZTYtOThkMS00M2I1LThlMWUtMjk3YmNiZGY0OWVh',
 'NTAyOTFiMjMtZTJlOS00MDI0LWEyYmMtNWM4NTQ5YWQwMjZh',
 'NmUxODY0MWUtNGM5My00MWM2LWFhZGUtY2M4MjUwODg4NDQ2',
 'ODEzZTM0ZWEtYWZjMy00MDk0LThmYWYtMTk5Mjc3ZjQ5NjUz',
 'YzYxMzc3MjEtN2I4NC00ZDk2LWFkNDEtZjNmMTFkYmY1ODk5',
 'ODJkNTQwMmUtNGU5Ni00MjA3LWI3MjUtMGQwYTU4ZDI5OWJh',
 'NGFjMjA1ZTYtYjY4Mi00NzNiLTgyNTMtZjNjOWU3Yzc2ZmZi',
 'YzNjZTM3YTMtNzk2ZC00ZTRkLTg4ZGQtMzBjZDFhNjQ1

## Retrive relevant chunks based on a question

In [34]:
# Retrieve relevant chunks based on the question

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.get_relevant_documents(
    "For AI wash what is suggested spin setting for washing?"
)

print(retrieved_docs[0].page_content)



| Program | Type of Laundry / Notes | Model / Max Load | Temperature Setting | Spin Setting | Selectable Options |
| --- | --- | --- | --- | --- | --- |
| BD-D120XGV/BD-D120GV | BD-D100XGV/BD-D100GV | Washing | Washing- Drying | U Prewash | Rinse Count | Rinse Hold | Timer | Auto Self Clean | Wash/Dry Mode | Dry Option | Time Dry |
| Washing | Washing- Drying | Drying | Washing | Washing- Drying | Drying | Washing | Washing- Drying | Drying |
| Tub Wash | This program is used when you are worried about dirt and odors on the drum. Please do not put laundry. | - | - :unselected: | - :unselected: | - :unselected: | - :unselected: | – :unselected: | – :unselected: | 1200 r/min | – :unselected: | - :unselected: | - :unselected: | - :unselected: | - :unselected: | - :unselected: | O :selected: | - :unselected: | - :unselected: | – :unselected: | – :unselected: |
| Rinse & Spin (\*) (When use BD-D120XGV/ BD-D100XGV Please download the program from the application.) | This program is used to

## Document Q&A

In [35]:
# Ask a question about the document
# Use a prompt for RAG that is checked into the LangChain prompt hub (https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=989ad331-949f-4bac-9694-660074a208a7)
prompt = hub.pull("rlm/rag-prompt")
llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",  # e.g., "2023-12-01-preview"
    azure_deployment="gpt-4-turbo",
    temperature=0,
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("can i repair the appliance on my own?")

'Based on the provided context, you should not attempt to repair the appliance on your own. The instructions explicitly advise against dismantling, repairing, or modifying the appliance, as it could lead to malfunction, fire, electric shock, or injury. For repairs, you are instructed to contact your local service center.'

## Doucment Q&A with references

In [36]:
# Return the retrieved documents or certain source metadata from the documents

from operator import itemgetter

from langchain.schema.runnable import RunnableMap

rag_chain_from_docs = (
    {
        "context": lambda input: format_docs(input["documents"]),
        "question": itemgetter("question"),
    }
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain_with_source = RunnableMap(
    {"documents": retriever, "question": RunnablePassthrough()}
) | {
    "documents": lambda input: [doc.metadata for doc in input["documents"]],
    "answer": rag_chain_from_docs,
}

rag_chain_with_source.invoke("can i repair the appliance on my own?")

{'documents': [{'Header 2': 'General safety'},
  {'Header 2': 'Customer Service'},
  {'Header 2': 'Installation'}],
 'answer': 'Based on the provided context, you should not attempt to repair the appliance on your own. The instructions explicitly advise against dismantling, repairing, or modifying the appliance, as it could lead to malfunction, fire, electric shock, or injury. For repairs, you are instructed to contact your local service center.'}

In [37]:
rag_chain_with_source.invoke("can i lift the appliance with one hand?")

{'documents': [{'Header 2': 'General safety'},
  {'Header 2': 'Safety Instructions'},
  {'Header 2': 'Interior lighting setting (For model BD-D120XGV/BD-D100XGV)',
   'Header 3': 'Emergency door release.'}],
 'answer': "The provided context does not specify the weight of the appliance or whether it is designed to be lifted with one hand. Therefore, based on the information given, I cannot determine if you can lift the appliance with one hand. It is generally recommended to follow the manufacturer's guidelines for moving appliances to avoid injury or damage."}

In [38]:
rag_chain_with_source.invoke("where is water supply hose located?")

{'documents': [{'header_text': ['Cover Caps',
    'Either type of water supply hose is packed in by the demand of different area.',
    'Hose Guide',
    'Packed-In Accessories',
    'Spanner'],
   'page': [3],
   'row_count': 3,
   'column_count': 5,
   'type': 'table'},
  {'Header 2': 'Hose and cable length'},
  {'Header 2': '1', 'Header 3': '2'}],
 'answer': 'The water supply hose is typically located on the left side of the appliance, as indicated by the provided context. It can be connected using either a one-touch joint or a screw type, depending on the region, and is included as a packed-in accessory with the appliance. To access the hose, you may need to disconnect it by turning the union nut in the direction indicated in the instructions.'}

In [39]:
rag_chain_with_source.invoke("How do i pair the appliance with phone?")

{'documents': [{'header_text': ['Check here', 'In this case'],
   'page': [60, 61, 62, 63],
   'row_count': 34,
   'column_count': 2,
   'type': 'table'},
  {'Header 2': '5\\. Proceed to connect (pairing) the washer dryer to your smartphone.'},
  {'header_text': ['Check here', 'In this case'],
   'page': [60, 61, 62, 63],
   'row_count': 34,
   'column_count': 2,
   'type': 'table'}],
 'answer': 'To pair the appliance with your phone, first ensure your smartphone is connected to the correct 2.4GHz Wi-Fi network. Then, follow the "Simple connection" process by operating the app on your smartphone, tapping "Pairing," selecting "Easy Connection," and following the on-screen instructions to connect the washer dryer to your Wi-Fi router. If "Pr 1" does not flash on the washer dryer, press the "Remote" button, and ensure the communication lamp on the washer dryer changes from flashing to lit to confirm successful pairing.'}

In [47]:
rag_chain_with_source.invoke("what is max load for AI wash program for washing and drying?")

{'documents': [{'header_text': ['Auto Self Clean',
    'BD-D100XGV/BD-D100GV',
    'BD-D120XGV/BD-D120GV',
    'Dry Option',
    'Drying',
    'Drying',
    'Drying',
    'Model / Max Load',
    'Program',
    'Rinse Count',
    'Rinse Hold',
    'Selectable Options',
    'Spin Setting',
    'Temperature Setting',
    'Time Dry',
    'Timer',
    'Type of Laundry / Notes',
    'U Prewash',
    'Wash/Dry Mode',
    'Washing',
    'Washing',
    'Washing',
    'Washing',
    'Washing- Drying',
    'Washing- Drying',
    'Washing- Drying',
    'Washing- Drying'],
   'page': [36, 37, 38, 39],
   'row_count': 24,
   'column_count': 21,
   'type': 'table'},
  {'Header 2': 'To set and unset fabric softener'},
  {'Header 2': 'Hints and Tips for Eco-friendly Washing'}],
 'answer': 'The max load for the AI Wash program for washing is 12 kg, and for drying, it is 8 kg. If drying at a low temperature, the max load is reduced to 4 kg.'}

In [49]:
rag_chain_with_source.invoke("what type of laundry i can use for AI wash program?")

{'documents': [{'Header 2': 'To set and unset fabric softener'},
  {'Header 2': 'Hints and Tips for Eco-friendly Washing'},
  {'Header 2': 'Operation:'}],
 'answer': 'The AI Wash program is designed for washing your usual laundry. It automatically adjusts the washing method and operating time based on various factors such as the type of detergent, type of clothing, water hardness, and load size. You should keep to the load limit specified for your model, which is 12 kg for model BD-D120XGV/BD-D120GV and 10 kg for model BD-D100XGV/BD-D100GV.'}

In [48]:
rag_chain_with_source.invoke("For AI wash program what is suggested spin setting for washing?")

{'documents': [{'header_text': ['Auto Self Clean',
    'BD-D100XGV/BD-D100GV',
    'BD-D120XGV/BD-D120GV',
    'Dry Option',
    'Drying',
    'Drying',
    'Drying',
    'Model / Max Load',
    'Program',
    'Rinse Count',
    'Rinse Hold',
    'Selectable Options',
    'Spin Setting',
    'Temperature Setting',
    'Time Dry',
    'Timer',
    'Type of Laundry / Notes',
    'U Prewash',
    'Wash/Dry Mode',
    'Washing',
    'Washing',
    'Washing',
    'Washing',
    'Washing- Drying',
    'Washing- Drying',
    'Washing- Drying',
    'Washing- Drying'],
   'page': [36, 37, 38, 39],
   'row_count': 24,
   'column_count': 21,
   'type': 'table'},
  {'header_text': ['Auto Self Clean',
    'BD-D100XGV/BD-D100GV',
    'BD-D120XGV/BD-D120GV',
    'Dry Option',
    'Drying',
    'Drying',
    'Drying',
    'Model / Max Load',
    'Program',
    'Rinse Count',
    'Rinse Hold',
    'Selectable Options',
    'Spin Setting',
    'Temperature Setting',
    'Time Dry',
    'Timer',
    'Typ

In [52]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retrieved_docs = retriever.get_relevant_documents(
    "For AI wash program what is suggested temperature setting?"
)

print(retrieved_docs)

[Document(page_content='Keep to the load limit of each program.  \nYour washer dryer automatically adjusts the operating time and amount of water of each program depending on the load.  \nYou only need to use the Prewash options if your laundry is heavily soiled.  \nAdd detergent according to the degree of soiling, laundry size and water hardness, and follow the detergent manufacturer\'s instructions.  \nUsing the Cotton program with water temperature at 60℃ setting instead of the Cotton program with water temperature at 90℃ will save energy, and will remove ordinary stains. Better still, in areas where the temperature is high, you can remove ordinary stains without using hot water by setting \'Cold\' programs for more economically friendly operation.  \n1 CAUTION  \n<figure>  \n![](figures/3)  \n</figure>  \nThis symbol shows that the Operation Manual should be read carefully.  \n<!-- PageNumber="2" -->  \n<!-- PageHeader="Parts and Accessories" -->', metadata={'Header 2': 'Hints and 

In [44]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.get_relevant_documents(
    "For AI wash what is suggested spin setting for washing?"
)

print(retrieved_docs)

[Document(page_content='\n\n| Program | Type of Laundry / Notes | Model / Max Load | Temperature Setting | Spin Setting | Selectable Options |\n| --- | --- | --- | --- | --- | --- |\n| BD-D120XGV/BD-D120GV | BD-D100XGV/BD-D100GV | Washing | Washing- Drying | U Prewash | Rinse Count | Rinse Hold | Timer | Auto Self Clean | Wash/Dry Mode | Dry Option | Time Dry |\n| Washing | Washing- Drying | Drying | Washing | Washing- Drying | Drying | Washing | Washing- Drying | Drying |\n| Tub Wash | This program is used when you are worried about dirt and odors on the drum. Please do not put laundry. | - | - :unselected: | - :unselected: | - :unselected: | - :unselected: | – :unselected: | – :unselected: | 1200 r/min | – :unselected: | - :unselected: | - :unselected: | - :unselected: | - :unselected: | - :unselected: | O :selected: | - :unselected: | - :unselected: | – :unselected: | – :unselected: |\n| Rinse & Spin (\\*) (When use BD-D120XGV/ BD-D100XGV Please download the program from the applica